In [1]:
import requests
import base64
import pandas as pd
import numpy as np

CLIENT_ID = 'fa9f3d44584944cf8cd7d988ffe47c6d'
CLIENT_SECRET = '2de72e647f51489b80408089da9080eb'

SPOTIFY_TOKEN_URL = "https://accounts.spotify.com/api/token"
SPOTIFY_SEARCH_URL = "https://api.spotify.com/v1/search"


def get_access_token(client_id, client_secret):
    client_creds = f"{client_id}:{client_secret}"
    client_creds_b64 = base64.b64encode(client_creds.encode())

    token_data = {
        'grant_type': 'client_credentials'
    }
    token_headers = {
        'Authorization': f'Basic {client_creds_b64.decode()}'
    }

    r = requests.post(SPOTIFY_TOKEN_URL, data=token_data,
                      headers=token_headers)
    token_response_data = r.json()
    
    return token_response_data.get('access_token')

access_token = get_access_token(CLIENT_ID, CLIENT_SECRET)

#### Fetch Albums

In [110]:
def fetch_albums(access_token, offset=0):
    headers = {'Authorization': f'Bearer {access_token}'}

    search_params = {
        'q': 'tag:new',
        'type': 'album',
        'limit': 50,
        'offset': offset
    }
    
    response = requests.get("https://api.spotify.com/v1/search", headers=headers, params=search_params)
    
    if response.ok:
        results = response.json()['albums']['items']
        return results

    return response.status_code

def get_album_details(access_token, album_ids:list):
    url = f"https://api.spotify.com/v1/albums?ids={','.join(album_ids)}"
    headers = {'Authorization': f'Bearer {access_token}'}
    response = requests.get(url, headers=headers)
    return response.json()


def get_artist_genres(access_token, artist_id):
    url = f"https://api.spotify.com/v1/artists/{artist_id}"
    headers = {'Authorization': f'Bearer {access_token}'}
    response = requests.get(url, headers=headers)
    return response.json().get('genres', [])


def get_album_artists(access_token, album_id):
    url = f"https://api.spotify.com/v1/albums/{album_id}"
    headers = {'Authorization': f'Bearer {access_token}'}
    response = requests.get(url, headers=headers)
    album_data = response.json()
    artist_ids = [artist['id'] for artist in album_data['artists']]
    return artist_ids

#### Fetch Songs

In [111]:
def fetch_songs(access_token, offset):
    headers = {'Authorization': f'Bearer {access_token}'}

    search_params = {
        # 'q': 'tag:new',
        'type': 'track',
        'limit': 50,
        'offset': offset
    }
    
    response = requests.get("https://api.spotify.com/v1/search", headers=headers, params=search_params)
    
    if response.ok:
        results = response.json()['tracks']['items']
        return results

    return response.status_code


def get_song_features(access_token, track_ids):
    features_url = f"https://api.spotify.com/v1/audio-features?ids={','.join(track_ids)}"
    header = {
        "Authorization": f"Bearer {access_token}"
    }

    res = requests.get(features_url, headers=header)
    return res.json()


def map_key_to_pitch_class(key):
    key_map = {
        -1: "",
        0: "C",
        1: "C♯, D♭",
        2: "D",
        3: "D♯, E♭",
        4: "E",
        5: "F",
        6: "F♯, G♭",
        7: "G",
        8: "G♯, A♭",
        9: "A",
        10: "A♯, B♭",
        11: "B"
    }
    return key_map.get(key, "")


def map_mode_to_name(mode):
    mode_map = {
        0: "Minor",
        1: "Major"
    }
    return mode_map.get(mode, "")

In [ ]:
import time
import math
import numpy as np

all_new_songs = pd.DataFrame()
offset = 0

while len(all_new_songs) < 2000:
    songs = fetch_songs(access_token, offset=offset)
    song_ids = [song["id"] for song in songs]

    song_details = []
    for chunk in [song_ids[x:x+20] for x in range(0, len(song_ids), 20)]:
        details = get_song_features(access_token, chunk)['audio_features']

        for i, song_id in enumerate(chunk):
            song_info = songs[i]
            detail = details[i]  

            # if song_info["popularity"] > 50: #long tail
            #     continue

            if detail is not None and song_info is not None:
                merged_song_detail = {**song_info, **detail}
                song_details.append(merged_song_detail)

    print(len(all_new_songs) + len(song_details))

    songs_df = pd.DataFrame([{
        'track_name': song.get('name', ''),
        'artist(s)_name': ", ".join([artist['name'] for artist in song['artists']]),
        'artist_count': len(song.get('artists')),
        'album': song.get('album', {}).get('name', ""),
        'explicit': song.get('explicit', ''),
        'popularity': song.get('popularity', np.NaN),
        'release_date': song.get('album', {}).get('release_date', ""),
        'streams': np.NaN,
        'duration_in_min': song.get('duration_ms', np.NaN) / 60000,
        'bpm': math.floor(song.get('tempo', np.NaN)),
        'key': map_key_to_pitch_class(song.get('key', -1)),
        'mode': map_mode_to_name(song.get('mode', np.NaN)),
        'danceability_%': math.ceil(song.get('danceability', np.NaN) * 100),
        'valence_%': math.ceil(song.get('valence', np.NaN) * 100),
        'energy_%': math.ceil(song.get('energy', np.NaN) * 100),
        'acousticness_%': math.ceil(song.get('acousticness', np.NaN) * 100),
        'instrumentalness_%': math.ceil(song.get('instrumentalness', np.NaN) * 100),
        'liveness_%': math.ceil(song.get('liveness', np.NaN) * 100),
        'speechiness_%': math.ceil(song.get('speechiness', np.NaN) * 100),
    } for song in song_details])

    all_new_songs = pd.concat(
        [all_new_songs, songs_df], ignore_index=True)

    time.sleep(1)
    offset += 50

In [ ]:
all_new_songs.head(n=50)

#### Get Data

In [ ]:
import time

import numpy as np

all_new_albums = pd.DataFrame()
offset = 0

song_ids = []

while len(all_new_albums) < 5000:
    new_albums = fetch_albums(access_token, offset=offset)

    new_album_ids = [album["id"] for album in new_albums]

    new_album_details = []
    for chunk in [new_album_ids[x:x+20] for x in range(0, len(new_album_ids), 20)]:
        album_details = get_album_details(access_token, chunk)['albums'] 
        
        for album in album_details:
            
            print(album)
            
            if album["album_type"] == "album":
                artist_ids = get_album_artists(access_token, album["id"])
                genres = []
                
                for artist_id in artist_ids:
                    genres += get_artist_genres(access_token, artist_id)
                    
                album["genres"] = list(set(genres))
                new_album_details.append(album)
                
                song_ids += [song["id"]for song in album["tracks"]["items"]]

    print(len(all_new_albums) + len(new_album_details))

    new_albums_df = pd.DataFrame([{
        'album_name': album['name'],
        'artist_name': ", ".join([artist["name"] for artist in album['artists']]),
        'release_date': album['release_date'],
        'genres': ", ".join(album['genres']),
        'descriptors': "",
        'avg_rating': np.NaN,
        'rating_count': np.NaN,
        'review_count': np.NaN,
        'popularity': album['popularity'],
        'total_tracks': album['total_tracks'],
    } for album in new_album_details])

    all_new_albums = pd.concat(
        [all_new_albums, new_albums_df], ignore_index=True)
    
    time.sleep(1)
    offset += 50

In [126]:
def fetch_song(access_token, song_id):
    url = f"https://api.spotify.com/v1/tracks/{song_id}"
    headers = {'Authorization': f'Bearer {access_token}'}
    response = requests.get(url, headers=headers)
    return response.json()

def get_song_features(access_token, track_ids):
    features_url = f"https://api.spotify.com/v1/audio-features?ids={','.join(track_ids)}"
    header = {
        "Authorization": f"Bearer {access_token}"
    }

    res = requests.get(features_url, headers=header)
    return res.json()

song_details = []
all_new_songs = pd.DataFrame()

for i, song_id in enumerate(song_ids):
    print(f"{i} of {len(song_ids)}")
    
    song_info = fetch_song(access_token, song_id)
    detail = get_song_features(access_token, [song_id])["audio_features"][0]
        
    if detail is not None and song_info is not None:
        merged_song_detail = {**song_info, **detail}
        song_details.append(merged_song_detail)

songs_df = pd.DataFrame([{
    'track_name': song.get('name', ''),
    'artist(s)_name': ", ".join([artist['name'] for artist in song.get('artists', [])]),
    'artist_count': len(song.get('artists')),
    'album': song.get('album', {}).get('name', ""),
    'explicit': song.get('explicit', ''),
    'popularity': song.get('popularity', np.NaN),
    'release_date': song.get('album', {}).get('release_date', ""),
    'streams': np.NaN,
    'duration_in_min': song.get('duration_ms', np.NaN) / 60000,
    'bpm': math.floor(song['tempo']) if 'tempo' in song and song['tempo'] is not None else np.NaN,
    'key': map_key_to_pitch_class(song.get('key', -1)),
    'mode': map_mode_to_name(song.get('mode', np.NaN)),
    'danceability_%': math.ceil(song.get('danceability', np.NaN) * 100) if 'danceability' in song and song['danceability'] is not None else np.NaN,
    'valence_%': math.ceil(song.get('valence', np.NaN) * 100) if 'valence' in song and song['valence'] is not None else np.NaN,
    'energy_%': math.ceil(song.get('energy', np.NaN) * 100) if 'energy' in song and song['energy'] is not None else np.NaN,
    'acousticness_%': math.ceil(song.get('acousticness', np.NaN) * 100) if 'acousticness' in song and song['acousticness'] is not None else np.NaN,
    'instrumentalness_%': math.ceil(song.get('instrumentalness', np.NaN) * 100) if 'instrumentalness' in song and song['instrumentalness'] is not None else np.NaN,
    'liveness_%': math.ceil(song.get('liveness', np.NaN) * 100) if 'liveness' in song and song['liveness'] is not None else np.NaN,
    'speechiness_%': math.ceil(song.get('speechiness', np.NaN) * 100) if 'speechiness' in song and song['speechiness'] is not None else np.NaN,
} for song in song_details])

all_new_songs = pd.concat(
    [all_new_songs, songs_df], ignore_index=True)

0 of 497
1 of 497
2 of 497
3 of 497
4 of 497
5 of 497
6 of 497
7 of 497
8 of 497
9 of 497
10 of 497
11 of 497
12 of 497
13 of 497
14 of 497
15 of 497
16 of 497
17 of 497
18 of 497
19 of 497
20 of 497
21 of 497
22 of 497
23 of 497
24 of 497
25 of 497
26 of 497
27 of 497
28 of 497
29 of 497
30 of 497
31 of 497
32 of 497
33 of 497
34 of 497
35 of 497
36 of 497
37 of 497
38 of 497
39 of 497
40 of 497
41 of 497
42 of 497
43 of 497
44 of 497
45 of 497
46 of 497
47 of 497
48 of 497
49 of 497
50 of 497
51 of 497
52 of 497
53 of 497
54 of 497
55 of 497
56 of 497
57 of 497
58 of 497
59 of 497
60 of 497
61 of 497
62 of 497
63 of 497
64 of 497
65 of 497
66 of 497
67 of 497
68 of 497
69 of 497
70 of 497
71 of 497
72 of 497
73 of 497
74 of 497
75 of 497
76 of 497
77 of 497
78 of 497
79 of 497
80 of 497
81 of 497
82 of 497
83 of 497
84 of 497
85 of 497
86 of 497
87 of 497
88 of 497
89 of 497
90 of 497
91 of 497
92 of 497
93 of 497
94 of 497
95 of 497
96 of 497
97 of 497
98 of 497
99 of 497
100 of 497

In [129]:
all_new_songs.head(n=10)

,track_name,artist(s)_name,artist_count,album,explicit,popularity,release_date,streams,duration_in_min,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,3D (feat. Jack Harlow),"Jung Kook, Jack Harlow",2,GOLDEN,True,85,2023-11-03,NaN,3.363533,108,C#,Major,86,89,83,4,0,9,11
1,Closer to You (feat. Major Lazer),"Jung Kook, Major Lazer",2,GOLDEN,False,86,2023-11-03,NaN,2.849917,113,D,Minor,79,50,66,12,1,11,5
2,Seven (feat. Latto) (Explicit Ver.),"Jung Kook, Latto",2,GOLDEN,True,87,2023-11-03,NaN,3.059183,124,B,Major,79,88,84,32,0,8,5
3,Standing Next to You,Jung Kook,1,GOLDEN,False,96,2023-11-03,NaN,3.433667,106,D,Minor,72,82,81,5,0,34,10
4,Yes or No,Jung Kook,1,GOLDEN,False,88,2023-11-03,NaN,2.459283,83,C#,Major,68,89,84,18,0,8,9
5,Please Don't Change (feat. DJ Snake),"Jung Kook, DJ Snake",2,GOLDEN,False,87,2023-11-03,NaN,2.444750,114,B,Major,80,77,75,17,1,17,18
6,Hate You,Jung Kook,1,GOLDEN,False,88,2023-11-03,NaN,2.570617,79,D,Major,42,25,28,87,0,10,4
7,Somebody,Jung Kook,1,GOLDEN,False,86,2023-11-03,NaN,2.810000,106,C#,Minor,65,26,68,29,0,14,5
8,Too Sad to Dance,Jung Kook,1,GOLDEN,False,86,2023-11-03,NaN,2.931867,100,A#,Major,56,57,47,62,0,11,9
9,Shot Glass of Tears,Jung Kook,1,GOLDEN,False,86,2023-11-03,NaN,2.787450,77,F#,Minor,51,17,40,51,0,9,4


In [130]:
all_new_songs.to_csv('./data/tracks_long_tail.csv', index=False)

In [ ]:
all_new_albums.head(n=10)

In [ ]:
niche_albums = pd.read_excel("./data/albums_niche.xlsx")

niche_albums['artist_name'] = niche_albums['artist_name'].str.lstrip('\n')
niche_albums['genres'] = niche_albums['genres'].str.lstrip('\n')
niche_albums['descriptors'] = niche_albums['descriptors'].str.lstrip('\n')
niche_albums['release_name'] = niche_albums['release_name'].str.lstrip('\n')

niche_albums['release_date'] = niche_albums['release_date'].apply(
    lambda x: str(x).replace(" 00:00:00", "") if "00:00:00" in str(x) else x)

niche_albums['descriptors'] = niche_albums['descriptors'].apply(
    lambda x: str(x).replace("\n", ", "))

niche_albums['genres'] = niche_albums['genres'].apply(
    lambda x: str(x).replace("\n", ", "))


niche_albums = niche_albums[niche_albums['descriptors'].notna()]

niche_albums.head(n=10)

In [ ]:
unpopular_albums = pd.read_excel("./data/unpopular_albums_2023.xlsx")

unpopular_albums['artist_name'] = unpopular_albums['artist_name'].str.lstrip('\n')
unpopular_albums['genres'] = unpopular_albums['genres'].str.lstrip('\n')
unpopular_albums['descriptors'] = unpopular_albums['descriptors'].str.lstrip('\n')
unpopular_albums['release_name'] = unpopular_albums['release_name'].str.lstrip('\n')

unpopular_albums['release_date'] = unpopular_albums['release_date'].apply(
    lambda x: str(x).replace(" 00:00:00", "") if "00:00:00" in str(x) else x)

unpopular_albums['descriptors'] = unpopular_albums['descriptors'].apply(
    lambda x: str(x).replace("\n", ", "))

unpopular_albums['genres'] = unpopular_albums['genres'].apply(
    lambda x: str(x).replace("\n", ", "))

unpopular_albums = unpopular_albums[unpopular_albums['descriptors'].notna()]

unpopular_albums.head(n=10)

In [ ]:
# merge them

merged_df = pd.concat([niche_albums, unpopular_albums], ignore_index=True)

unpopular_albums.to_csv('./data/albums_long_tail.csv', index=False)

### Preprocess Data

In [74]:
# df_albums = pd.read_csv("albums_popular.csv")
# df_albums = df_albums.drop(df_albums.columns[[0]], axis=1)


# df_albums.to_csv("albums_popular.csv")

# df_albums.head()

,release_name,artist_name,release_date,release_type,primary_genres,secondary_genres,descriptors,avg_rating,rating_count,review_count
0,OK Computer,Radiohead,1997-06-16,album,"Alternative Rock, Art Rock",NaN,"melancholic, anxious, futuristic, malevocals, ...",4.24,74027,1541
1,Kid A,Radiohead,2000-10-03,album,"Art Rock, Experimental Rock, Electronic","Ambient, Electronic, IDM","cold, melancholic, futuristic, anxious, atmosp...",4.23,61658,751
2,The Dark Side of the Moon,Pink Floyd,1973-03-23,album,"Art Rock, Progressive Rock","Psychedelic Rock, Space Rock","philosophical, atmospheric, introspective, exi...",4.21,60192,1557
3,Loveless,My Bloody Valentine,1991-11-11,album,"Shoegaze, Noise Pop","Dream Pop, Neo-Psychedelia","noisy, ethereal, atmospheric, romantic, love, ...",4.24,53174,1264
4,My Beautiful Dark Twisted Fantasy,Kanye West,2010-11-22,album,"Pop Rap, Hip Hop",Art Pop,"epic, boastful, passionate, sampling, hedonist...",4.09,52149,638


In [75]:
for file_name in "albums_long_tail.csv", "albums_niche.xlsx", "albums_popular.csv", "albums_unpopular_2023.xlsx":

    # columns_to_load = ['release_name', 'artist_name', 'release_date', 'genres', 'descriptors', 'avg_rating', 'rating_count', 'review_count']

    
    if file_name.endswith(".csv"):
        df = pd.read_csv(file_name)
    else:
        df = pd.read_excel(file_name)

    print(f"Column headers in {file_name}:")
    print(df.columns.tolist())
    print("\n") 

Column headers in albums_long_tail.csv:
['release_name', 'artist_name', 'release_date', 'genres', 'descriptors', 'avg_rating', 'rating_count', 'review_count']


Column headers in albums_niche.xlsx:
['release_name', 'artist_name', 'release_date', 'genres', 'descriptors', 'avg_rating', 'rating_count', 'review_count']


Column headers in albums_popular.csv:
['Unnamed: 0', 'release_name', 'artist_name', 'release_date', 'release_type', 'primary_genres', 'secondary_genres', 'descriptors', 'avg_rating', 'rating_count', 'review_count']


Column headers in albums_unpopular_2023.xlsx:
['release_name', 'artist_name', 'release_date', 'genres', 'descriptors', 'avg_rating', 'rating_count', 'review_count']




In [273]:
import re

def preprocess(file_path:str, usecols):
    if file_path.endswith(".csv"):
        df_albums = pd.read_csv(file_path)
    else:
        df_albums = pd.read_excel(file_path)

    # release_name
    df_albums['release_name'] = df_albums['release_name'].astype(str).str.strip()

    # artist_name
    df_albums['artist_name'] = df_albums['artist_name'].astype(str).apply(
        lambda x: re.sub(r"[\n\t]", "", x).strip()
    )


    def to_datetime(date):
        """target format: YYYY-MM-DD"""
        match = re.search("\d{4}-\d{2}-\d{2}", date)
        if match:
            return match.group()

        match = re.search("\d{4}-\d{2}", date)
        if match:
            return match.group() + "-01"

        match = re.search("\d{4}", date)
        if match:
            return match.group() + "-01-01"

    # release_date
    df_albums["release_date"] = pd.to_datetime(df_albums["release_date"].astype(str).apply(
        lambda x: to_datetime(x)
    ))

    # genres
    def combine_genres(row):
        primary_genres = row['primary_genres'] if pd.notna(
            row['primary_genres']) else ''
        secondary_genres = row['secondary_genres'] if pd.notna(
            row['secondary_genres']) else ''

        primary_genres_list = [genre.strip()
                            for genre in primary_genres.split(',')]
        secondary_genres_list = [genre.strip()
                                for genre in secondary_genres.split(',')]

        combined_genres = list(dict.fromkeys(
            primary_genres_list + secondary_genres_list))

        combined_genres = [genre for genre in combined_genres if genre]

        return ', '.join(combined_genres)

    if "secondary_genres" in usecols:
        df_albums['genres'] = df_albums.apply(combine_genres, axis=1)
        df_albums = df_albums.drop(['primary_genres', 'secondary_genres'], axis=1)

        columns = df_albums.columns.tolist()
        release_date_index = columns.index('release_date') + 1
        columns.remove('genres')
        columns.insert(release_date_index, 'genres')
        df_albums = df_albums[columns]


    # descriptors
    df_albums['descriptors'] = df_albums['descriptors'].astype(str).apply(
        lambda x: re.sub(r"[\n\t]", "", x).strip()
    )
    
    df_albums["review_count"] = df_albums["review_count"].fillna(0).astype(int)
    
    return df_albums

In [274]:
usecols = ['release_name', 'artist_name', 'release_date',
           'primary_genres', 'secondary_genres', 'descriptors', 'avg_rating', 'rating_count', 'review_count']

usecols2 = ['release_name', 'artist_name', 'release_date',
            'descriptors', 'avg_rating', 'rating_count', 'review_count']

albums_popular = preprocess("albums_popular.csv", usecols)

albums_popular.head(n=5)

,Unnamed: 0,release_name,artist_name,release_date,genres,release_type,descriptors,avg_rating,rating_count,review_count
0,0,OK Computer,Radiohead,1997-06-16,"Alternative Rock, Art Rock",album,"melancholic, anxious, futuristic, malevocals, ...",4.24,74027,1541
1,1,Kid A,Radiohead,2000-10-03,"Art Rock, Experimental Rock, Electronic, Ambie...",album,"cold, melancholic, futuristic, anxious, atmosp...",4.23,61658,751
2,2,The Dark Side of the Moon,Pink Floyd,1973-03-23,"Art Rock, Progressive Rock, Psychedelic Rock, ...",album,"philosophical, atmospheric, introspective, exi...",4.21,60192,1557
3,3,Loveless,My Bloody Valentine,1991-11-11,"Shoegaze, Noise Pop, Dream Pop, Neo-Psychedelia",album,"noisy, ethereal, atmospheric, romantic, love, ...",4.24,53174,1264
4,4,My Beautiful Dark Twisted Fantasy,Kanye West,2010-11-22,"Pop Rap, Hip Hop, Art Pop",album,"epic, boastful, passionate, sampling, hedonist...",4.09,52149,638


In [275]:
albums_long_tail = preprocess("albums_long_tail.csv", usecols2)
albums_long_tail.head(n=10)

,release_name,artist_name,release_date,genres,descriptors,avg_rating,rating_count,review_count
0,EndEx,3TEETH,2023-09-22,"Industrial Metal, Industrial Rock,Dark Electro...","futuristic, male vocalist, martial, dark, scie...",3.31,141,3
1,Chill Kill,Red Velvet,2023-11-13,"K-Pop, Contemporary R&B, Alt-Pop, Trap [EDM], ...","female vocalist, dark, nocturnal, ethereal, se...",3.40,510,2
2,Un/limited Love,George Riley,2023-11-10,"Alternative R&B, Electronic Dance Music,UK Gar...","female vocalist, sexual, sensual, ethereal, en...",3.45,55,1
3,Sweet Justice,Tkay Maidza,2023-11-03,"Contemporary R&B, Pop Rap, Hip House, Industri...","female vocalist, nocturnal, bittersweet, eclec...",3.38,670,3
4,Drama,aespa,2023-11-10,"K-Pop, Trap [EDM], Dance-Pop, Electropop, Danc...","female vocalist, energetic, eclectic, playful,...",2.84,528,1
5,Heaven Knows,PinkPantheress,2023-11-10,"Contemporary R&B, Alt-Pop, Electronic Dance Mu...","female vocalist, atmospheric, ethereal, bitter...",3.66,2869,23
6,I<3UQTINVU,Jockstrap & Taylor Skye,2023-11-03,"Glitch Pop, Electronic Dance Music, House, Hyp...","female vocalist, surreal, sampling, LGBT, play...",3.01,466,6
7,Blanket,Kevin Abstract,2023-11-03,"Indie Rock, Indie Pop,Bedroom Pop, Slacker Rock","abstract, male vocalist, LGBT, introspective, ...",2.80,1030,13
8,going…going…GONE!,hemlocke springs,2023-09-29,"Synthpop, Bedroom Pop,New Wave, Dance-Pop, Ind...","quirky, longing, playful, female vocalist, lov...",3.70,1108,6
9,Ambrosia,Namasenda,2023-10-05,"Alternative R&B,Bubblegum Bass, Funk brasileir...","female vocalist, ethereal, rhythmic, cold, sen...",3.02,229,0


In [276]:
albums_nice = preprocess("albums_niche.xlsx", usecols2)
albums_nice.head(n=10)

,release_name,artist_name,release_date,genres,descriptors,avg_rating,rating_count,review_count
0,Night of the Spectre,Chaino,1958-01-01,Exotica,"playful,repetitive,rhythmic,mysterious,minimal...",3.25,33,2
1,The Chantels,We Are The Chantels,1958-01-01,"Girl Group, Doo-Wop","passionate, sentimental, vocal group, female v...",3.36,110,4
2,Lightnin' Hopkins,Lightnin' and the Blues,1959-01-01,"Electric Texas Blues, Blues","longing, rhythmic, raw, mellow, introspective,...",3.78,193,6
3,Ritchie Valens,Ritchie Valens,1959-02-12,"Rock & Roll, Tex-Mex, Traditional Pop, Hispani...","playful, mellow, ballad, anthemic, love, energ...",3.56,513,13
4,Rock!,Los Locos del Ritmo,1959-01-01,"Rock & Roll, Rockabilly","male vocalist, lo-fi, melodic, rhythmic, rebel...",3.25,63,2
5,الجمعة الحزينة Good Friday: Eastern Sacred Songs,Fairuz,1967-01-01,Syriac Chant,"Christian, female vocalist, religious, male vo...",3.54,374,5
6,Lord My Cell Is Cold / One More Day and One Mo...,Kack Klick,1964-01-01,"Garage Rock, Proto-Punk, Blues Rock, Blues","crime, raw, male vocalist, introspective, long...",3.47,73,2
7,Viens sur la montagne,Marie Laforêt,1964-01-01,"French Pop, Chanson, Contemporary Folk, Baroqu...","mellow, female vocalist, peaceful, ballad, mel...",3.58,82,2
8,96 Tears,Question Mark and The Mysterians,1966-01-01,"Garage Rock, Psychedelic Rock, British Rhythm ...","male vocalist, rhythmic, playful, love, breaku...",3.26,703,18
9,Voices Green and Purple / Trip to New Orleans,The Bees,1966-01-01,"Garage Rock, Psychedelic Rock, Acid Rock, Prot...","male vocalist, raw, dark, anxious, lo-fi, psyc...",3.81,177,7


In [293]:
albums = pd.concat([albums_popular, albums_long_tail, albums_nice], ignore_index=True)

albums = albums.loc[:, ~albums.columns.str.contains('^Unnamed')]

albums.head()

albums.to_csv("albums.csv", index=False)

In [294]:
songs_short_tail = pd.read_csv("./tracks.csv", encoding_errors="ignore")

# track_name
songs_short_tail['track_name'] = songs_short_tail['track_name'].astype(
    str).str.strip()

# artist_name
songs_short_tail['artist(s)_name'] = songs_short_tail['artist(s)_name'].astype(str).apply(
    lambda x: re.sub(r"[\n\t]", "", x).strip()
)


# one attribute for release_date
songs_short_tail['release_date'] = pd.to_datetime(songs_short_tail['released_year'].astype(str) + '-' +
                                                  songs_short_tail['released_month'].astype(str) + '-' +
                                                  songs_short_tail['released_day'].astype(str))

songs_short_tail.drop(['released_year', 'released_month',
                       'released_day'], axis=1, inplace=True)

songs_short_tail['release_date'] = pd.to_datetime(songs_short_tail['release_date'].apply(
    lambda x: str(x).replace("00:00:00", "") if "00:00:00" in str(x) else x))

# reorder:
new_order = ['track_name', 'artist(s)_name', 'artist_count', 'release_date', 'streams', 'bpm', 'key', 'mode',
             'danceability_%', 'valence_%', 'energy_%', 'acousticness_%', 'instrumentalness_%', 'liveness_%', 'speechiness_%']
songs_short_tail = songs_short_tail[new_order]

songs_short_tail.head()

,track_name,artist(s)_name,artist_count,release_date,streams,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023-07-14,141381703,125,B,Major,80,89,83,31,0,8,4
1,LALA,Myke Towers,1,2023-03-23,133716286,92,C#,Major,71,61,74,7,0,10,4
2,vampire,Olivia Rodrigo,1,2023-06-30,140003974,138,F,Major,51,32,53,17,0,31,6
3,Cruel Summer,Taylor Swift,1,2019-08-23,800840817,170,A,Major,55,58,72,11,0,11,15
4,WHERE SHE GOES,Bad Bunny,1,2023-05-18,303236322,144,A,Minor,65,23,80,14,63,11,6


In [295]:
songs_long_tail = pd.read_csv("./tracks_long_tail.csv", encoding_errors="ignore")

# track_name
songs_long_tail['track_name'] = songs_long_tail['track_name'].astype(
    str).str.strip()

# artist_name
songs_long_tail['artist(s)_name'] = songs_long_tail['artist(s)_name'].astype(str).apply(
    lambda x: re.sub(r"[\n\t]", "", x).strip()
)

songs_long_tail['release_date'] = pd.to_datetime(songs_long_tail['release_date'].apply(
    lambda x: str(x).replace("00:00:00", "") if "00:00:00" in str(x) else x))

songs_long_tail.head()

,track_name,artist(s)_name,artist_count,album,explicit,popularity,release_date,streams,duration_in_min,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,3D (feat. Jack Harlow),"Jung Kook, Jack Harlow",2,GOLDEN,True,85,2023-11-03,NaN,3.363533,108,C#,Major,86,89,83,4,0,9,11
1,Closer to You (feat. Major Lazer),"Jung Kook, Major Lazer",2,GOLDEN,False,86,2023-11-03,NaN,2.849917,113,D,Minor,79,50,66,12,1,11,5
2,Seven (feat. Latto) (Explicit Ver.),"Jung Kook, Latto",2,GOLDEN,True,87,2023-11-03,NaN,3.059183,124,B,Major,79,88,84,32,0,8,5
3,Standing Next to You,Jung Kook,1,GOLDEN,False,96,2023-11-03,NaN,3.433667,106,D,Minor,72,82,81,5,0,34,10
4,Yes or No,Jung Kook,1,GOLDEN,False,88,2023-11-03,NaN,2.459283,83,C#,Major,68,89,84,18,0,8,9


In [296]:
songs = pd.concat([songs_short_tail, songs_long_tail], ignore_index=True)

songs.head()

songs.to_csv("songs.csv", index=False)